In [1]:
from utils.consts import *

from utils.utilities import (
    get_json_data, get_subjs, get_agg_input_vector,
    get_where_cond_val, get_cond_op_input_vector)

# from agg_predict import predict_agg
from op_predict import predict_cond_op

import pickle
import json
from pprint import pprint
import tensorflow as tf
import numpy as np

In [2]:
import spacy
PIPE = spacy.load('en_core_web_lg')
# PIPE = spacy.load('en_coref_lg')

In [3]:
def process_question(question):
    doc = PIPE(question)
    subjs = get_subjs(doc)
    sel_col = subjs[0]
    X_agg = get_agg_input_vector(subjs[0], doc)
    
    cond_val = get_where_cond_val(doc)
    print("COND VAL : ", cond_val)
    X_cond_op = get_cond_op_input_vector(doc)
    return (X_agg, X_cond_op)

In [4]:
data = get_json_data('./data_with_num/dev_dataset.json')

In [10]:
q1 = data[0]['question']

In [11]:
q1

'How many schools did player number 3 play at?'

In [12]:
(q1_agg, q1_op) = process_question(q1)

COND VAL :  3


In [13]:
len(q1_op)

1200

In [ ]:
predict_cond_op(q1_op)

In [ ]:
predict_cond_op(q1_op)

In [8]:
def predict_op(cond_op_X):
    n_input = 1200
    n_hidden1 = 512
    n_hidden2 = 256
    n_hidden3 = 64
    n_output = 4

    learning_rate = 1e-4
    n_iteration = 31
    batch_size = 128


    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_output])
    # keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

    weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
    }

    biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
    }

    layer_1 = tf.nn.tanh(tf.add(tf.matmul(X, weights['w1']), biases['b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
    # layer_drop = tf.nn.dropout(layer_3, keep_prob)
    output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(layer_3, weights['out']), biases['out']))

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    tf.train.Saver().restore(sess, './checkpoints/op/training.ckpt')
    prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [cond_op_X]})
    return int(np.squeeze(prediction))

In [11]:
predict_agg(q1_x)

INFO:tensorflow:Restoring parameters from ./checkpoints/agg/training.ckpt


2

import tensorflow as tf
import numpy as np
import pickle
class AggNN():
    def __init__(self):
        n_input = 1500
        n_hidden1 = 1024
        n_hidden2 = 512
        n_hidden3 = 64
        n_output = 6
        
        self.learning_rate = 1e-4
        self.n_iteration = 101
        self.batch_size = 128
        
        self.X = tf.placeholder("float", [None, n_input])
        self.Y = tf.placeholder("float", [None, n_output])
        
        self.weights = {
            'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
            'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
            'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
            'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
        }
        
        self.biases = {
            'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
            'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
            'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
            'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
        }
        
        self.layer_1 = tf.nn.tanh(tf.add(tf.matmul(self.X, self.weights['w1']), self.biases['b1']))
        self.layer_2 = tf.nn.tanh(tf.add(tf.matmul(self.layer_1, self.weights['w2']), self.biases['b2']))
        self.layer_3 = tf.nn.relu(tf.add(tf.matmul(self.layer_2, self.weights['w3']), self.biases['b3']))
        # layer_drop = tf.nn.dropout(layer_3, keep_prob)
        self.output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(self.layer_3, self.weights['out']), self.biases['out']))
        
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=self.output_layer))
        self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cross_entropy)
        
        self.correct_pred = tf.equal(tf.argmax(self.output_layer, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
        
        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init)
        
        self.dev_X, self.dev_Y = self.get_data('dev')
        self.n_batches = len(self.dev_X)//self.batch_size
        
    def get_data(self, dt):
        x_file = './data/agg_data/%s_agg_X.pkl'
        y_file = './data/agg_data/%s_agg_Y.pkl'
        with open(y_file%dt, 'rb') as fy:
            _Y = pickle.load(fy)

        with open(x_file%dt, 'rb') as fx:
            _X = pickle.load(fx)
        return (_X, _Y)
    
    def get_batch(self, idx, batch_size):
        lw = idx * batch_size
        hs = lw + batch_size
        if hs < len(self.dev_X):
            return (self.dev_X[lw:hs][:], self.dev_Y[lw:hs][:])
        else:
            return (self.dev_X[lw:][:], self.dev_Y[lw:][:])
    
    def load_param(self):
        tf.train.Saver().restore(self.sess, './checkpoints/agg/training.ckpt')
        
    def store_param(self):
        tf.train.Saver().save(self.sess, './checkpoints/agg/training.ckpt')
        
    def test(self):
        test_X, test_Y = get_data('test')
        test_accuracy = sess.run(self.accuracy, feed_dict={self.X: test_X, self.Y: test_Y})
        print("\nAccuracy on test set : ", test_accuracy)
        
    def train(self):
        # train on mini batches
        for i in range(self.n_iteration):
            for j in range(self.n_batches+1):
                (batch_x, batch_y) = self.get_batch(j, self.batch_size)
                self.sess.run(self.train_step, feed_dict={self.X: batch_x, self.Y: batch_y})

                # print loss and accuracy (per minibatch)
            if i % 5 == 0:
                minibatch_loss, minibatch_accuracy = \
                    self.sess.run([self.cross_entropy, self.accuracy], feed_dict={self.X: batch_x, self.Y: batch_y})
                print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
    
    def predict(self, agg_X):
        prediction = self.sess.run(tf.argmax(self.output_layer,1), feed_dict={self.X: [agg_X]})
        return np.squeeze(prediction)

In [15]:
agg_nn = AggNN()

In [ ]:
agg_nn.load_param()